In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx
import optuna
import re

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer



from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('TARGET')
modules = Modules('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')

In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [4]:
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

In [5]:
apps, prev, bureau, bureau_bal, pos_bal, install, card_bal = FeatureEngineering3.get_dataset()
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df = FeatureEngineering3.get_apps_all_with_all_agg(apps, prev, bureau, bureau_bal, pos_bal, install, card_bal)
df.head()

prev_agg shape: (338857, 41) bureau_agg shape: (305811, 103)
pos_bal_agg shape: (337252, 45) install_agg shape: (339587, 59) card_bal_agg shape: (103558, 70)
apps_all before merge shape: (356255, 152)
apps_all after merge with all shape: (356255, 466)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [8]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [10]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [11]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 603 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 593 features are remained after removing non-informative features
60 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 533 features are remained after removing features not interesting for LightGBM classifier
---=> 38 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 571 features

---=> df final shape: (356250, 571)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,1,0,0,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,1,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,1,0,0,0,0,0,0,0,0


In [12]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 571 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 567 features are remained after removing non-informative features
22 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 545 features are remained after removing features not interesting for LightGBM classifier
---=> 15 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 560 features

---=> df final shape: (356250, 560)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [14]:
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,1,1,0,0,0,0,0,0
307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,1,1,0,0,0,0,0,0
307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,0,1,0,1,0,1,0,1,0
307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,0,1,1,1,0,1,0,1,0
307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [16]:
df_train = df_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,1,1,0,0,0,0,0,0,0.051055
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,1,1,0,0,0,0,0,0,0.166067
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,1,0,0.043899
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,1,1,0,1,0,1,0,0.056542
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0.183168


In [17]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.051055
1,100005,0.166067
2,100013,0.043899
3,100028,0.056542
4,100038,0.183168


In [18]:
df_train = df_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,1,0,0,0,0,0,0,0.051055,0.038227
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,1,0,0,0,0,0,0,0.166067,0.112768
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,1,0,0.043899,0.033638
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,1,0,1,0,1,0,0.056542,0.031213
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0.183168,0.181109


In [19]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.025023
1,100005,0.090335
2,100013,0.037033
3,100028,0.036644
4,100038,0.172674


In [20]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,0,0,0,0,0,0,0.051055,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,0,0,0,0,0,0,0.166067,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,0.043899,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,0,1,0,1,0,0.056542,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0.183168,0.181109,0.172674


In [21]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,1,0,0,0,0,0,0,0.516856,0.162368,0.156048
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0.010645,0.027249,0.012034
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0.054077,0.030869,0.026333
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,0,1,0,1,0,0.059492,0.029480,0.029856
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,0,0,0,0,0,0.076165,0.047569,0.034698


In [22]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,0,0,0,0,0,0,0.051055,0.038227,0.025023
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,0,0,0,0,0,0,0.166067,0.112768,0.090335
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,0.043899,0.033638,0.037033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,0,1,0,1,0,0.056542,0.031213,0.036644
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0.183168,0.181109,0.172674


In [23]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [24]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,1,0,0,0,0,0,0,0.516856,0.162368,0.156048
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,0,0,0,0,0,0,0.010645,0.027249,0.012034
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0,0,0,0,0,0,0,0.054077,0.030869,0.026333
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0,1,0,1,0,1,0,0.059492,0.029480,0.029856
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0,0,0,0,0,0,0,0.076165,0.047569,0.034698


In [25]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,5.053162,0.986709,0.860543
100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.839862,-0.548398,-0.659948
100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.334248,-0.507271,-0.508977
100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.271209,-0.523054,-0.471789
100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.077109,-0.317542,-0.420660


In [26]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,5.053162,0.986709,0.860543
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.839862,-0.548398,-0.659948
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.334248,-0.507271,-0.508977
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.271209,-0.523054,-0.471789
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.077109,-0.317542,-0.420660


In [27]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 695.61 MB
Memory usage of optimization is 692503640.00 MB
Decreased by -99552732.0%
Memory usage of dataframe is 110.17 MB
Memory usage of optimization is 109576640.00 MB
Decreased by -99459773.0%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a,set2a
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,0.645020,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,5.054688,0.986816,0.860352
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,0.645020,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.839844,-0.548340,-0.660156
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,-1.549805,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.334229,-0.507324,-0.508789
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,-1.549805,1.693359,-0.138672,1.614258,0.0,1.614258,0.0,-0.271240,-0.522949,-0.471680
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,-1.549805,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.077087,-0.317627,-0.420654


In [28]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [29]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [30]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 561) (61502, 561)
[100]	training's auc: 0.795698	valid_1's auc: 0.782806
[200]	training's auc: 0.80174	valid_1's auc: 0.786691
[300]	training's auc: 0.806482	valid_1's auc: 0.78891
[400]	training's auc: 0.811014	valid_1's auc: 0.790621
[500]	training's auc: 0.81487	valid_1's auc: 0.791869
[600]	training's auc: 0.81853	valid_1's auc: 0.79258
[700]	training's auc: 0.821979	valid_1's auc: 0.793313
[800]	training's auc: 0.82529	valid_1's auc: 0.793753
[900]	training's auc: 0.828262	valid_1's auc: 0.794143
[1000]	training's auc: 0.831191	valid_1's auc: 0.794359
[1100]	training's auc: 0.834081	valid_1's auc: 0.794527
[1200]	training's auc: 0.836685	valid_1's auc: 0.794627
[1300]	training's auc: 0.839362	valid_1's auc: 0.79473
[1400]	training's auc: 0.841834	valid_1's auc: 0.794726
[1500]	training's auc: 0.844286	valid_1's auc: 0.794689
[1600]	training's auc: 0.84665	valid_1's auc: 0.794755
[1700]	training's auc: 0.848867	valid_1's auc: 0.7

[600]	training's auc: 0.81769	valid_1's auc: 0.797682
[700]	training's auc: 0.821169	valid_1's auc: 0.79826
[800]	training's auc: 0.824447	valid_1's auc: 0.798675
[900]	training's auc: 0.827504	valid_1's auc: 0.798922
[1000]	training's auc: 0.830476	valid_1's auc: 0.799205
[1100]	training's auc: 0.833298	valid_1's auc: 0.799381
[1200]	training's auc: 0.836046	valid_1's auc: 0.799414
[1300]	training's auc: 0.83876	valid_1's auc: 0.799431
[1400]	training's auc: 0.841307	valid_1's auc: 0.799467
[1500]	training's auc: 0.843786	valid_1's auc: 0.799394
[1600]	training's auc: 0.846252	valid_1's auc: 0.799306
[1700]	training's auc: 0.848603	valid_1's auc: 0.79922
[1800]	training's auc: 0.850901	valid_1's auc: 0.799095
[1900]	training's auc: 0.853074	valid_1's auc: 0.799112
[2000]	training's auc: 0.855168	valid_1's auc: 0.799046
[2100]	training's auc: 0.857208	valid_1's auc: 0.798944
[2200]	training's auc: 0.859246	valid_1's auc: 0.798902
[2300]	training's auc: 0.861228	valid_1's auc: 0.798801


In [31]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
560,set2a,624.6,86.736959
232,DAYS_BIRTH,547.8,87.995454
21,APPS_ANNUITY_CREDIT_RATIO,474.4,27.978563
74,BUREAU_ACT_BUREAU_CREDIT_ENDDATE_DIFF_MAX,422.6,72.358828
559,pred_1b,393.8,59.246941
1,AMT_CREDIT,382.4,51.091095
46,APP_SCORE2_TO_CITY_RATING_RATIO,381.8,58.074952
120,BUREAU_BUREAU_CREDIT_DEBT_DIFF_MEAN,377.6,71.974301
252,EXT_SOURCE_2,376.0,60.807072
558,pred_1a,357.6,43.038355


In [32]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [33]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [34]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_OS_3-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.043545
1,100005,0.099132
2,100013,0.062258
3,100028,0.034962
4,100038,0.192450


## パラメータ・チューニング

In [35]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [36]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [37]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [38]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [39]:
### 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_OS_3-2.csv", index=None)